# (EX) Electronic medical record (as a SQL refresher)
This example utilizes a subset of the [MIMIC III demo](https://physionet.org/content/mimiciii-demo/1.4/) dataset in illustrating the use of database using DuckDB.

*Quick notes about DuckDB:*  

- DuckDB is a relational database for analytics processing (i.e., OLAP)
- DuckDB is columnar-oriented
- DuckDB scales reasonably for *relatively* large datasets and works well for local development

For MIMIC-III, here is a [full summary](https://mit-lcp.github.io/mimic-schema-spy/) where you can understand the relationship between tables.

In [1]:
#!pip install duckdb==1.2.2
import duckdb

In [3]:
# establish connection
conn = duckdb.connect('dataset/mimic.db', read_only=True)

conn.sql('SHOW TABLES;')

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ ADMISSIONS │
│ DRGCODES   │
│ D_ICDPROCS │
│ ICUSTAYS   │
│ PATIENTS   │
│ PROCS_ICD  │
└────────────┘

In [4]:
# metadata for a particular table
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns
    WHERE table_name = 'ADMISSIONS';
    """
)

┌──────────────────────┬───────────┐
│     column_name      │ data_type │
│       varchar        │  varchar  │
├──────────────────────┼───────────┤
│ row_id               │ INTEGER   │
│ subject_id           │ INTEGER   │
│ hadm_id              │ INTEGER   │
│ admittime            │ VARCHAR   │
│ dischtime            │ VARCHAR   │
│ deathtime            │ VARCHAR   │
│ admission_type       │ VARCHAR   │
│ admission_location   │ VARCHAR   │
│ discharge_location   │ VARCHAR   │
│ insurance            │ VARCHAR   │
│ language             │ VARCHAR   │
│ religion             │ VARCHAR   │
│ marital_status       │ VARCHAR   │
│ ethnicity            │ VARCHAR   │
│ edregtime            │ VARCHAR   │
│ edouttime            │ VARCHAR   │
│ diagnosis            │ VARCHAR   │
│ hospital_expire_flag │ INTEGER   │
│ has_chartevents_data │ INTEGER   │
├──────────────────────┴───────────┤
│ 19 rows                2 columns │
└──────────────────────────────────┘

In [22]:
# metadata for a particular table
conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns
    WHERE table_name = 'PATIENTS';
    """
)

conn.sql(
    """
    SELECT column_name, data_type FROM information_schema.columns
    WHERE table_name = 'DRGCODES';
    """
)

┌───────────────┬───────────┐
│  column_name  │ data_type │
│    varchar    │  varchar  │
├───────────────┼───────────┤
│ row_id        │ BIGINT    │
│ subject_id    │ BIGINT    │
│ hadm_id       │ BIGINT    │
│ drg_type      │ VARCHAR   │
│ drg_code      │ VARCHAR   │
│ description   │ VARCHAR   │
│ drg_severity  │ BIGINT    │
│ drg_mortality │ BIGINT    │
└───────────────┴───────────┘

In [5]:
# admission table at a glance
conn.sql(
    """
    SELECT deathtime FROM ADMISSIONS;
    """
)

┌─────────────────────┐
│      deathtime      │
│       varchar       │
├─────────────────────┤
│ NULL                │
│ 2126-08-28 18:59:00 │
│ 2125-10-07 15:13:00 │
│ NULL                │
│ 2163-05-15 12:00:00 │
│ NULL                │
│ NULL                │
│ NULL                │
│ NULL                │
│ NULL                │
│  ·                  │
│  ·                  │
│  ·                  │
│ NULL                │
│ 2152-10-09 22:33:00 │
│ NULL                │
│ NULL                │
│ NULL                │
│ NULL                │
│ 2178-05-15 09:45:00 │
│ NULL                │
│ NULL                │
│ NULL                │
├─────────────────────┤
│ 129 rows (20 shown) │
└─────────────────────┘

In [6]:
# number of patients
conn.sql(
    """
    SELECT COUNT(*) FROM PATIENTS;
    """
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          100 │
└──────────────┘

## Refresher for simple queries

1. How many records are in each of the available tables?
2. How many patients are female?
3. How many patients passed away during the hospital stay?
4. How many different admission types are there? What are they?
5. What is the earliest and the latest admission time in the database?

In [21]:
# 1. How many records are in each of the available tables?
conn.sql(
    """
    SELECT 'ADMISSIONS' AS table_name, COUNT(*) AS count FROM ADMISSIONS
    UNION ALL
    SELECT 'DRGCODES', COUNT(*) FROM DRGCODES
    UNION ALL
    SELECT 'D_ICDPROCS', COUNT(*) FROM D_ICDPROCS
    UNION ALL
    SELECT 'ICUSTAYS', COUNT(*) FROM ICUSTAYS
    UNION ALL
    SELECT 'PATIENTS', COUNT(*) FROM PATIENTS
    UNION ALL
    SELECT 'PROCS_ICD', COUNT(*) FROM PROCS_ICD;
    """
    )

# 2. How many patients are female?
conn.sql(
    """
    SELECT COUNT(*) FROM PATIENTS
    WHERE gender = 'F';
    """
)

# 3. How many patients passed away during the hospital stay?
conn.sql(
    """
    SELECT COUNT(*) FROM PATIENTS
    WHERE dod_hosp IS NOT NULL;
    """
)

# 4. How many different admission types are there? What are they?
conn.sql(
    """
    SELECT DISTINCT admission_type FROM ADMISSIONS;
    """
)

# 5. What is the earliest and the latest admission time in the database?
conn.sql(
    """
    SELECT MIN(admittime), MAX(admittime) FROM ADMISSIONS;
    """
)


┌─────────────────────┬─────────────────────┐
│   min(admittime)    │   max(admittime)    │
│       varchar       │       varchar       │
├─────────────────────┼─────────────────────┤
│ 2102-08-29 07:15:00 │ 2202-10-03 01:45:00 │
└─────────────────────┴─────────────────────┘

## Slightly more complicated queries
1. Create a table with all ICU stays with their respective patient information.
2. Create a table to show all unique DRG (diagnosis-related group) codes and the number of associated admissions.

In [25]:
# 1. Create a table with all ICU stays with their respective patient information.
conn.sql(
    """
    SELECT * FROM ICUSTAYS JOIN ADMISSIONS ON ICUSTAYS.hadm_id = ADMISSIONS.hadm_id;
    """
)


# 2. Create a table to show all unique DRG (diagnosis-related group) codes and the number of associated admissions.
conn.sql(
    """
    SELECT drg_code, COUNT(DISTINCT hadm_id) AS num_admissions FROM DRGCODES
    GROUP BY drg_code
    ORDER BY num_admissions DESC;
    """
)
## ordered the table by descending to ensure codes were grouped together! (a lot of codes w/ 1 admin)

┌──────────┬────────────────┐
│ drg_code │ num_admissions │
│ varchar  │     int64      │
├──────────┼────────────────┤
│ 7204     │             12 │
│ 871      │             11 │
│ 416      │              8 │
│ 7104     │              4 │
│ 1394     │              4 │
│ 208      │              3 │
│ 1304     │              3 │
│ 207      │              3 │
│ 001      │              2 │
│ 6814     │              2 │
│  ·       │              · │
│  ·       │              · │
│  ·       │              · │
│ 190      │              1 │
│ 6914     │              1 │
│ 2072     │              1 │
│ 314      │              1 │
│ 565      │              1 │
│ 4423     │              1 │
│ 2213     │              1 │
│ 0203     │              1 │
│ 218      │              1 │
│ 518      │              1 │
├──────────┴────────────────┤
│    162 rows (20 shown)    │
└───────────────────────────┘

## Derivative queries

1. Calculate the age of each patient at the time of admission.  
    *Hint:* `cast(patients.dob as date)` allows for addition and subtraction of dates (in days)
2. Identify if a patient passes away when they are in the ICU.
3. Calculate the average duration of admission.

In [43]:
# Calculate the age of each patient at the time of admission.
  # Hint: cast(patients.dob as date) allows for addition and subtraction of dates (in days)
conn.sql(
    """
    SELECT p.subject_id, a.hadm_id, CAST(a.admittime AS DATE) AS admittime_date, CAST(p.dob AS DATE) AS dob_date,
    ROUND((CAST(a.admittime AS DATE) - CAST(p.dob AS DATE)) / 365.25, 1) AS age_at_admission,
    FROM PATIENTS p
    JOIN ADMISSIONS a
    ON p.subject_id = a.subject_id;
    """
)



┌────────────┬─────────┬────────────────┬────────────┬──────────────────┐
│ subject_id │ hadm_id │ admittime_date │  dob_date  │ age_at_admission │
│   int32    │  int32  │      date      │    date    │      double      │
├────────────┼─────────┼────────────────┼────────────┼──────────────────┤
│      10006 │  142345 │ 2164-10-23     │ 2094-03-05 │             70.6 │
│      10011 │  105331 │ 2126-08-14     │ 2090-06-05 │             36.2 │
│      10013 │  165520 │ 2125-10-04     │ 2038-09-03 │             87.1 │
│      10017 │  199207 │ 2149-05-26     │ 2075-09-21 │             73.7 │
│      10019 │  177759 │ 2163-05-14     │ 2114-06-20 │             48.9 │
│      10026 │  103770 │ 2195-05-17     │ 1895-05-17 │            300.0 │
│      10027 │  199395 │ 2190-07-13     │ 2108-01-15 │             82.5 │
│      10029 │  132349 │ 2139-09-22     │ 2061-04-10 │             78.4 │
│      10032 │  140372 │ 2138-04-02     │ 2050-03-29 │             88.0 │
│      10033 │  157235 │ 2132-12-05   